In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

## Parsing the Excel file

The first header's row only contains the timestamp starting from 00:00 with an increment of 15 minutes.

We can ignore it since we can re-calculate the value from the id of velocity and remove that header increase in the readability of the table.

In [2]:
df = pd.read_excel("data/Scats Data October 2006.xls", sheet_name="Data", header=[1])
df

,SCATS Number,Location,CD_MELWAY,NB_LATITUDE,NB_LONGITUDE,HF VicRoads Internal,VR Internal Stat,VR Internal Loc,NB_TYPE_SURVEY,Date,...,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95
0,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,2006-10-01 00:15:00,...,114,97,97,66,81,50,59,47,29,34
1,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,2006-10-02 00:15:00,...,111,102,107,114,80,60,62,48,44,26
2,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,2006-10-03 00:15:00,...,130,132,114,86,93,90,73,57,29,40
3,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,2006-10-04 00:15:00,...,115,113,132,101,113,90,78,66,52,44
4,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,2006-10-05 00:15:00,...,171,120,116,113,99,91,61,55,49,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,2006-10-27 00:15:00,...,122,121,127,103,122,124,117,99,108,88
4188,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,2006-10-28 00:15:00,...,93,93,93,105,105,112,82,97,106,107
4189,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,2006-10-29 00:15:00,...,87,118,83,76,66,64,77,60,49,45
4190,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,2006-10-30 00:15:00,...,90,88,89,80,74,48,67,62,50,62


In [34]:
first_velo_col_pos = df.columns.get_loc("V00")
flow_group = np.char.mod("V%02d", np.arange(0, 96))
grouped = df.groupby(['NB_LATITUDE', 'NB_LONGITUDE'])[flow_group].apply(lambda x: x.values.tolist())

In [35]:
flow_data = grouped.values
flow_scaler = np.array(flow_data.max()).max()

In [5]:
latlong_data = np.array(grouped.index.to_list())
latlong_data

array([[-37.8676   , 145.09146  ],
       [-37.86735  , 145.09195  ],
       [-37.86723  , 145.09103  ],
       [-37.86703  , 145.09159  ],
       [-37.86155  , 145.05751  ],
       [-37.86152  , 145.05851  ],
       [-37.8612671, 145.058038 ],
       [-37.86088  , 145.05744  ],
       [-37.85513  , 145.09376  ],
       [-37.85489  , 145.09413  ],
       [-37.85467  , 145.09384  ],
       [-37.85221  , 145.09425  ],
       [-37.85193  , 145.09463  ],
       [-37.85187  , 145.09407  ],
       [-37.8516827, 145.0943457],
       [-37.84741  , 145.05263  ],
       [-37.84726  , 145.05308  ],
       [-37.84714  , 145.05205  ],
       [-37.84686  , 145.04508  ],
       [-37.84683  , 145.05275  ],
       [-37.84632  , 145.04378  ],
       [-37.83799  , 145.09681  ],
       [-37.83777  , 145.06115  ],
       [-37.83756  , 145.06057  ],
       [-37.83755  , 145.09741  ],
       [-37.83741  , 145.09626  ],
       [-37.83738  , 145.06119  ],
       [-37.83725  , 145.06055  ],
       [-37.83695  ,

In [59]:

train = []

lags = 7
train = []
# for data in flow_values:
# 	indices = np.arange(lags, len(data))
# 	offset = np.arange(-lags, 1)
# 	flow_arr = data[indices[:, np.newaxis] + offset]
# 	# latlong_arr = np.tile(latlong_data, (len(flow_arr), 1))
# 	# print(flow_arr.shape)
# 	print(latlong_data.shape)
# 	# combined_arr = np.hstack((latlong_data_tiled, flow_arr))
# 	# train.extend(combined_arr)
# # train
# # np.array(train)[:, -1]

# flow_data

i = 0
for flow in grouped.values:
	flow = np.array(flow, dtype=float).flatten() / flow_scaler
	indices = np.arange(lags, len(flow))
	offset = np.arange(-lags, 1)
	flow = flow[indices[:, np.newaxis] + offset]
	latlong = np.tile(latlong_data[i], (len(flow), 1))
	combined_arr = np.hstack((latlong, flow))
	train.extend(combined_arr)
	i += 1
	# print(latlong.shape)
np.array(train)[:, -1]

array([0.05660377, 0.05345912, 0.05660377, ..., 0.04402516, 0.0408805 ,
       0.03459119])

In [19]:
len(np.unique(df["Location"]))
len((df["Date"]))

# np.column_stack((df["NB_LATITUDE"], df["NB_LONGITUDE"]))

0      2006-10-01 00:15:00
1      2006-10-02 00:15:00
2      2006-10-03 00:15:00
3      2006-10-04 00:15:00
4      2006-10-05 00:15:00
               ...        
4187   2006-10-27 00:15:00
4188   2006-10-28 00:15:00
4189   2006-10-29 00:15:00
4190   2006-10-30 00:15:00
4191   2006-10-31 00:15:00
Name: Date, Length: 4192, dtype: datetime64[ns]

In [4]:
min(np.unique(np.array(df['Date'])))

numpy.datetime64('2006-10-01T00:15:00.000000000')

## Getting the positon of the first collums of velocity

Return the array of arrays containing velocity from V00 to V95

In [5]:

first_velo_col_pos = df.columns.get_loc("V00")
flow_data = df.to_numpy()[:, first_velo_col_pos:]
flow_data

array([[86, 83, 52, ..., 47, 29, 34],
       [32, 28, 17, ..., 48, 44, 26],
       [26, 32, 21, ..., 57, 29, 40],
       ...,
       [100, 81, 89, ..., 60, 49, 45],
       [40, 29, 36, ..., 62, 50, 62],
       [36, 30, 24, ..., 63, 51, 54]], dtype=object)

In [6]:
# flatten_flow_data = flow_data.flatten().reshape(-1, 1)
# flow_scaler = MinMaxScaler(feature_range=(0, 1)).fit(flatten_flow_data)
# flow_values = flow_scaler.transform(flatten_flow_data)
# flow_values
flow_scaler = MinMaxScaler(feature_range=(0, 1)).fit(flow_data)
flow_values = flow_scaler.transform(flow_data)
flow_values.reshape(-1, 1)

array([[0.28289474],
       [0.32936508],
       [0.25870647],
       ...,
       [0.28506787],
       [0.1827957 ],
       [0.25471698]])

In [7]:
flatten_flow_data = flow_data.reshape(-1, 1)
flow_scaler = MinMaxScaler(feature_range=(0, 1)).fit(flatten_flow_data)
flow_values = flow_scaler.transform(flatten_flow_data)

In [8]:
lat_data = df['NB_LATITUDE'].to_numpy().reshape(-1, 1)
long_data = df['NB_LONGITUDE'].to_numpy().reshape(-1, 1)
np.concatenate((lat_data, long_data), axis=1)

array([[-37.86703, 145.09159],
       [-37.86703, 145.09159],
       [-37.86703, 145.09159],
       ...,
       [-37.81296, 145.0083 ],
       [-37.81296, 145.0083 ],
       [-37.81296, 145.0083 ]])

In [9]:
latlong_data = np.concatenate((lat_data, long_data), axis=1)
latlong_scaler = MinMaxScaler(feature_range=(0, 1)).fit(latlong_data)
latlong = latlong_scaler.transform(latlong_data)
latlong


array([[1.50524459e-05, 9.99949965e-01],
       [1.50524459e-05, 9.99949965e-01],
       [1.50524459e-05, 9.99949965e-01],
       ...,
       [1.44292218e-03, 9.99375943e-01],
       [1.44292218e-03, 9.99375943e-01],
       [1.44292218e-03, 9.99375943e-01]])

In [10]:
num_time_steps = 96

In [60]:
time_values = np.arange(num_time_steps) * 15 / 24 / 60
time_column = np.tile(time_values, len(flow_data)).reshape(-1, 1)
time_column

array([[0.        ],
       [0.01041667],
       [0.02083333],
       ...,
       [0.96875   ],
       [0.97916667],
       [0.98958333]])

In [13]:
expanded_latlong = np.repeat(latlong, num_time_steps, axis=0).reshape(-1, 2)
expanded_latlong

array([[1.50524459e-05, 9.99949965e-01],
       [1.50524459e-05, 9.99949965e-01],
       [1.50524459e-05, 9.99949965e-01],
       ...,
       [1.44292218e-03, 9.99375943e-01],
       [1.44292218e-03, 9.99375943e-01],
       [1.44292218e-03, 9.99375943e-01]])

In [14]:
# shifted_flow_values = np.roll(flow_values, -1, axis=1)
# shifted_flow_column = shifted_flow_values.reshape(-1, 1)
# shifted_flow_column

shifted_flow_column = np.roll(flow_values, -1)
shifted_flow_column

array([[0.11942446],
       [0.07482014],
       [0.08345324],
       ...,
       [0.07338129],
       [0.07769784],
       [0.12374101]])

In [61]:
train = np.hstack((time_column, expanded_latlong, shifted_flow_column))
train

array([[0.00000000e+00, 1.50524459e-05, 9.99949965e-01, 1.19424460e-01],
       [1.04166667e-02, 1.50524459e-05, 9.99949965e-01, 7.48201439e-02],
       [2.08333333e-02, 1.50524459e-05, 9.99949965e-01, 8.34532374e-02],
       ...,
       [9.68750000e-01, 1.44292218e-03, 9.99375943e-01, 7.33812950e-02],
       [9.79166667e-01, 1.44292218e-03, 9.99375943e-01, 7.76978417e-02],
       [9.89583333e-01, 1.44292218e-03, 9.99375943e-01, 1.23741007e-01]])

In [ ]:
X = train[:, :-1]
y = train[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)

(X_train, y_train, X_test, y_test)

(array([[1.16035290e+18, 1.40489495e-03, 9.99873190e-01],
        [1.16225370e+18, 1.92486453e-03, 9.99740522e-01],
        [1.16216730e+18, 1.94202960e-03, 9.99793244e-01],
        ...,
        [1.16182170e+18, 7.81934952e-04, 9.99985941e-01],
        [1.16121690e+18, 1.12338780e-03, 9.99856305e-01],
        [1.16087130e+18, 1.17910826e-03, 9.99484283e-01]]),
 array([0.00863309, 0.0057554 , 0.00719424, ..., 0.32661871, 0.19856115,
        0.2057554 ]),
 array([[1.16035290e+18, 1.72574972e-03, 9.99737834e-01],
        [1.15974810e+18, 2.25820226e-03, 9.99855103e-01],
        [1.16087130e+18, 1.72284486e-03, 9.99743761e-01],
        ...,
        [1.15992090e+18, 9.67845863e-04, 9.99544655e-01],
        [1.16087130e+18, 9.67845863e-04, 9.99544655e-01],
        [1.16052570e+18, 1.67134965e-03, 9.99883666e-01]]),
 array([0.00431655, 0.21438849, 0.13956835, ..., 0.01294964, 0.13381295,
        0.2028777 ]))